In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import stumpy
import pandas as pd
from scipy import signal
from sklearn.linear_model import Ridge
from scipy.spatial import distance
from fatf.utils.kernels import exponential_kernel 

import warnings
warnings.filterwarnings('ignore')

import importlib
import sys
sys.path.append('../')
from Utils.explanationsnew import LIMESegment, NEVES, LEFTIST, NNSegment, RBP, background_perturb
from Utils.data import loadUCRDataID
from Utils.models import *
from Utils.metrics import *

In [2]:
dataset_map = [('Coffee', 0),
                ('Strawberry', 1),
                   ('GunPointOldVersusYoung', 2),
                   ('HandOutlines', 3),
                    ('yoga', 4),
                    ('ECG200', 5),
                    ('GunPointMaleVersusFemale', 6),
                    ('DodgerLoopGame', 7),
                    ('Chinatown', 8),
                    ('FreezerSmallTrain', 9),
                    ('HouseTwenty', 10),
                    ('WormsTwoClass', 11)
                    ]

In [3]:
datasets = {}
for data_idx in dataset_map:
    datasets[data_idx[0]] = loadUCRDataID(data_idx[1])

Loading train / test dataset :  ../Data/Coffee_TRAIN ../Data/Coffee_TEST
Loading train / test dataset :  ../Data/Strawberry_TRAIN ../Data/Strawberry_TEST
Loading train / test dataset :  ../Data/GunPointOldVersusYoung_TRAIN ../Data/GunPointOldVersusYoung_TEST
Loading train / test dataset :  ../Data/HandOutlines_TRAIN ../Data/HandOutlines_TEST
Loading train / test dataset :  ../Data/Yoga_TRAIN ../Data/Yoga_TEST
Loading train / test dataset :  ../Data/ECG200_TRAIN ../Data/ECG200_TEST
Loading train / test dataset :  ../Data/GunPointMaleVersusFemale_TRAIN ../Data/GunPointMaleVersusFemale_TEST
Loading train / test dataset :  ../Data/DodgerLoopGame_TRAIN ../Data/DodgerLoopGame_TEST
Loading train / test dataset :  ../Data/Chinatown_TRAIN ../Data/Chinatown_TEST
Loading train / test dataset :  ../Data/FreezerSmallTrain_TRAIN ../Data/FreezerSmallTrain_TEST
Loading train / test dataset :  ../Data/HouseTwenty_TRAIN ../Data/HouseTwenty_TEST
Loading train / test dataset :  ../Data/WormsTwoClass_TRAIN

In [4]:
"""Datasets['Coffee'] : Train x, Train y, Test x, Test y

Train x : N x T x 1
Train y: N x 1 
Test x: N' x T x 1
Test y: N' x 2"""

"Datasets['Coffee'] : Train x, Train y, Test x, Test y\n\nTrain x : N x T x 1\nTrain y: N x 1 \nTest x: N' x T x 1\nTest y: N' x 2"

In [5]:
for data_idx in datasets.keys():
    print(datasets[data_idx][0].shape)

(28, 286, 1)
(613, 235, 1)
(136, 150, 1)
(1000, 2709, 1)
(300, 426, 1)
(100, 96, 1)
(135, 150, 1)
(20, 288, 1)
(20, 24, 1)
(28, 301, 1)
(40, 2000, 1)
(181, 900, 1)


In [6]:
BATCH_SIZES = [8, 64, 32, 64, 64, 16, 16, 4, 4, 4, 8, 16]
WINDOW_SIZES = [20, 20, 10, 100, 50, 10, 10, 20, 3, 10, 200, 100]
CPS = [5, 5, 4, 8, 5, 3, 4, 5, 2, 5, 8, 6]

In [7]:
models = ['knn','cnn','lstmfcn']
trained_models = {}
i = 0
for data_idx in datasets.keys():
    print(data_idx)
    trained_models[data_idx] = {}
    trained_models[data_idx]['knn'] = train_KNN_model(datasets[data_idx][0],datasets[data_idx][1])
    model_cnn = make_CNN_model(datasets[data_idx][0].shape[1:])
    trained_models[data_idx]['cnn'] = train_CNN_model(model_cnn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])[0]
    model_lstmfcn = make_LSTMFCN_model(datasets[data_idx][0].shape[1])
    trained_models[data_idx]['lstmfcn'] = train_LSTMFCN_model(model_lstmfcn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      datasets[data_idx][2],
                                                      datasets[data_idx][3],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])
    i = i + 1

Coffee
Epoch 1/2


2024-11-06 21:32:57.527292: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 1s 99ms/step - loss: 0.7265 - sparse_categorical_accuracy: 0.4545 - val_loss: 0.7397 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
3/3 [==============================] - 0s 11ms/step - loss: 0.6607 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.7468 - val_sparse_categorical_accuracy: 0.0000e+00
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 286, 1)]     0           []                               
                                                                                                  
 permute (Permute)              (None, 1, 286)       0           ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 

2024-11-06 21:32:58.149980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-06 21:32:58.150578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-06 21:32:58.151150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/2


...

Epoch 1/2


2024-11-06 21:33:38.411767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-06 21:33:38.412411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-06 21:33:38.412858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 - 2s - loss: 0.8755 - accuracy: 0.5359 - val_loss: 0.6917 - val_accuracy: 0.5195 - lr: 0.0010 - 2s/epoch - 165ms/step
Epoch 2/2


2024-11-06 21:33:40.159718: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-06 21:33:40.160349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-06 21:33:40.160969: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 - 1s - loss: 0.5407 - accuracy: 0.7072 - val_loss: 0.6608 - val_accuracy: 0.5974 - lr: 0.0010 - 857ms/epoch - 71ms/step


In [8]:
"""model_KNN shape : N x T 
## model_CNN/LSTMFCN shape  : N x T x 1


SyntaxError: incomplete input (3663073547.py, line 1)

In [9]:
MODEL_TYPES = ['class','proba','proba']

In [10]:
def reshaper(x,j):
    if j == 0:
        return x.reshape(x.shape[0])
    else:
        return x
#print(datasets.keys())
#print(datasets['Coffee'][2][0].shape)
#print(reshaper(datasets[data_idx][2][0],0))

In [11]:
#RBP([0,1,0],datasets['Coffee'][2],[0,300,600,-1],1)

In [12]:
explanations = {}
i = 0 
noisy_explanations = {} # For Robustness later
for data_idx in datasets.keys():
    print('processing explanations for: ' + str(data_idx) + '\n')
    explanations[data_idx] = {}
    noisy_explanations[data_idx] = {}
    j = 0
    for model_idx in trained_models[data_idx].keys():
        print('processing explanations for: ' + str(model_idx) + '\n')
        explanations[data_idx][model_idx] = {}
        noisy_explanations[data_idx][model_idx] = {}
        explanation_set = datasets[data_idx][2][0:2]
        explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), explanation_set, trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in explanation_set]
        explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in explanation_set]
        explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in explanation_set]
        
        noisy_set = np.asarray([add_noise(x) for x in explanation_set])
        
        noisy_explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in noisy_set]
        noisy_explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in noisy_set]
        noisy_explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in noisy_set]
        
        j = j + 1
    i = i + 1

processing explanations for: Coffee

processing explanations for: knn



ValueError: could not broadcast input array from shape (37,) into shape (2,286,1)

In [ ]:
evaluation_metrics = {}
for data_idx in datasets.keys():
    evaluation_metrics[data_idx] = {}
    j = 0 
    for model_idx in trained_models[data_idx].keys():
        evaluation_metrics[data_idx][model_idx] = {}
        for explanation_idx in explanations[data_idx][model_idx].keys():
            evaluation_metrics[data_idx][model_idx][explanation_idx] = {}
            # Robustness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Robustness'] = robustnessy(explanations[data_idx][model_idx][explanation_idx],
                                                                                         noisy_explanations[data_idx][model_idx][explanation_idx])
            explanation_set = datasets[data_idx][2][0:2]
            explanation_labels = datasets[data_idx][3][0:2]
            if j == 0:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set.reshape(explanation_set.shape[:2]))
            
            else:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set)
                
                # Faithfulness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Faithfulness'] = faithfulnessy(explanations[data_idx][model_idx][explanation_idx],explanation_set,explanation_labels,explanation_predictions,trained_models[data_idx][model_idx],model_type=MODEL_TYPES[j])
        j+=1
    